# Mysql & Python para Análise de dados
Usando MySQL Connector e Python para implementar um banco de dados no MySQL Server e para criar, ler, atualizar e excluir dados nesse banco de dados.

--------------
## Introdução
Este notebook foi desenvolvido para fins de aprendizado.

O notebook leva o leitor passo a passo por todos os processos envolvidos com o uso do Python e do MySQL Connector para executar as funções CRUD padrão em um banco de dados executado no MySQL Server e extração de dados via API do Twitter.

--------------

## Métodos usados
*Defining functions in Python\
*Database Implementation\
*Creating, Reading, Updating and Deleting data using SQL and Python

## Tecnologias usadas
*MySQL Community Server\
*MySQL Python Connector\
*Jupyter Notebook\
*pandas

--------------

## Fontes de dados
Os dados utilizados foram fornecidos para resolução de um case técnico.

## 1. Importação de Bibliotecas
Nosso primeiro passo é importar Mysql connector and pandas.

-----------------

In [1]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

--------------- 

## 2. Conexão com o servidor e criação do Banco de dados

### 2.1 - Definição da função de conexão com servidor.
Em seguida, queremos definir uma função em python que se conecte ao nosso MySQL Server. Para fazer isso, usamos o método mysql.connector.connect() .

Atualize a variável 'pw' com a senha de root do seu MySQL Server! Caso contrário, a conexão não pode ser feita.

In [30]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            passwd=''
        )
        print("Conexao com banco Mysql concluída")
    except Error as err:
        print(f"Error: '{err}'")

    return connection
pw = "" # IMPORTANTE! Coloque a senha do seu servidor Mysql aqui.
db = "db_botica" # Esse é o nome do banco de dados que criaremos em nossa próxima etapa. Chame como quiser.

connection = create_server_connection("localhost", "root", pw)

Conexao com banco Mysql concluída


--------------
#####  2.2 Criaçao de um banco de dados

Agora definimos uma função para criar um novo banco de dados em nosso servidor. Aqui estamos usando [cursor.execute()](https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-execute.html) para executar o comando [CREATE DATABASE](https://dev.mysql.com/doc/refman/8.0/en/creating-database.html).

In [31]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database criado com sucesso")
    except Error as err:
        print(f"Error: '{err}'")

create_database_query = "CREATE DATABASE db_botica"
create_database(connection, create_database_query)

Database criado com sucesso


-------------

#### 2.3 - Modificar a função de conexão do servidor, criar a função de conexão do banco de dados

Agora que criamos um banco de dados, vamos modificar nossa função create_server_connection para criar uma nova função para conectar diretamente a esse banco de dados. Isso será mais útil do que apenas conectar-se ao nosso servidor.

In [32]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            database='db_botica'
        )
        print("Conexão com banco Mysql concluída")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

----------
#### 2.4 - Definir Função de Execução de Consulta

O passo final desta etapa é criar uma função que nos permitirá executar consultas escritas em SQL. Isso vai ser extremamente útil!

Novamente, usamos [cursor.execute()](https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-execute.html) para executar nossos comandos.


In [33]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)#
        connection.commit()
        print("Query executada")
    except Error as err:
        print(f"Error: '{err}'")

-------------------

### 3. Leitura dos arquivos

#### 3.1 - Importação dos arquivos
Agora vamos importar os arquivos que vamos usar e tranforma-los em um [Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
, usando nossas funções recém-definidas. Importante lembrar que os arquivos estão no mesmo local do notebook.



In [2]:
df1 = pd.read_excel('Base_2017.xlsx')
df2 = pd.read_excel('Base_2018.xlsx')
df3 = pd.read_excel('Base_2019.xlsx')

-------------

#### 3.2 - Concatenação dos três arquivos.

Unimos os três dataframes que possuem os mesmos numeros de colunas e linhas, criando assim uma nova tabela com os dados das tres tabelas.xlsx.


In [35]:
total_vendas = pd.concat([df1,df2,df3],axis=0, ignore_index=True, join='outer')

In [36]:
total_vendas.head()

,ID_MARCA,MARCA,ID_LINHA,LINHA,DATA_VENDA,QTD_VENDA
0,4,VULT,2,PERFUMARIA,2019-01-15,7
1,4,VULT,2,PERFUMARIA,2019-10-12,2
2,3,QUEM DISSE BERENICE,5,HIDRATANTES,2019-09-01,3
3,3,QUEM DISSE BERENICE,3,MAQUIAGEM,2019-07-29,17
4,1,BOTICÁRIO,4,SOLAR,2019-11-15,19


--------------------------
#### 3.3 - Formataçao dos nomes das colunas

As colunas que antes eram maiúsculas passam a ser minúsculas.


In [37]:
total_vendas.rename(columns={'ID_MARCA': 'id_marca', 'MARCA':'marca', 'ID_LINHA':'id_linha', 'LINHA':'linha','DATA_VENDA':'dt_venda','QTD_VENDA':'qtd_vendas'}, inplace = True)

In [38]:
total_vendas.head()

,id_marca,marca,id_linha,linha,dt_venda,qtd_vendas
0,4,VULT,2,PERFUMARIA,2019-01-15,7
1,4,VULT,2,PERFUMARIA,2019-10-12,2
2,3,QUEM DISSE BERENICE,5,HIDRATANTES,2019-09-01,3
3,3,QUEM DISSE BERENICE,3,MAQUIAGEM,2019-07-29,17
4,1,BOTICÁRIO,4,SOLAR,2019-11-15,19


In [39]:
total_vendas.shape

(3000, 6)

In [41]:
total_vendas

,id_marca,marca,id_linha,linha,dt_venda,qtd_vendas
0,4,VULT,2,PERFUMARIA,2019-01-15,7
1,4,VULT,2,PERFUMARIA,2019-10-12,2
2,3,QUEM DISSE BERENICE,5,HIDRATANTES,2019-09-01,3
3,3,QUEM DISSE BERENICE,3,MAQUIAGEM,2019-07-29,17
4,1,BOTICÁRIO,4,SOLAR,2019-11-15,19
...,...,...,...,...,...,...
2995,1,BOTICÁRIO,2,PERFUMARIA,2019-11-19,14
2996,5,BELEZA NA WEB,2,PERFUMARIA,2019-12-14,5
2997,2,EUDORA,4,SOLAR,2019-10-03,9
2998,3,QUEM DISSE BERENICE,3,MAQUIAGEM,2019-03-03,20


## 4. Criando a tabela.

### 4.1 - Criação da tabela com total de vendas.

Crio a primeira tabela dentro de nosso banco de dados, usando nossas funções ja definidas.

In [43]:
create_vendas_table = """
CREATE TABLE total_vendas(
    id_marca INT,
    marca TEXT,
    id_linha INTEGER,
    linha TEXT,
    dt_venda date,
    qtd_vendas Int );
"""

connection = create_db_connection('localhost', "root", pw, db) # Conexao com banco de dados
execute_query(connection, create_vendas_table) # Execuçao da query 

Conexão com banco Mysql concluída
Query executada


### 4.2  - Leitura, população e  exportação dos dados.

Através de um loop, todas linhas do dataframe sao lidos e exportados para o banco de dados.



In [ ]:
for i, DADOS  in enumerate (total_vendas['qtd_vendas']):
    id_marca = total_vendas.loc[i,'id_marca']
    marca  = total_vendas.loc[i,'marca']
    id_linha = total_vendas.loc[i,'id_linha']
    linha = total_vendas.loc[i,'linha']
    dt_venda = total_vendas.loc[i,'dt_venda']
    qtd_vendas = total_vendas.loc[i,'qtd_vendas']
    
    connection = create_db_connection('localhost', "root", pw, db)
    dados = '('+'\''+str(id_marca)+'\''+',\''+str(marca)+'\''+',\''+str(id_linha)+'\''+',\''+str(linha)+'\''+',\''+str(dt_venda)+'\''+',\''+str(qtd_vendas)+'\''+')'
    inserir_no_banco = """INSERT INTO `total_vendas`(`id_marca`, `marca`, `id_linha`, `linha`, `dt_venda`, `qtd_vendas`) VALUES"""
    comando = inserir_no_banco + dados    
    
    try:
        execute_query(connection, comando)
    except:
        continue

# 4.3 - Criação das tabelas de visualização com as informações.

Para consumo e visualização por outra área, foi necessário modelar 4 novas tabelas e implementar processos 
que façam as transformações necessárias e insiram as seguintes visões nas tabelas:

Tabela1: Consolidado de vendas por ano e mês;
Tabela2: Consolidado de vendas por marca e linha;
Tabela3: Consolidado de vendas por marca, ano e mês;
Tabela4: Consolidado de vendas por linha, ano e mês;

In [45]:
create_consolidado_vendas_ano_mes = """
CREATE VIEW Tabela1 AS
SELECT SUM(qtd_vendas) AS total_de_vendas, 
DATE_FORMAT(dt_venda, '%Y-%m') AS Ano_mes
FROM total_vendas 
GROUP BY YEAR(dt_venda), MONTH(dt_venda)
ORDER BY MONTH(dt_venda); 
"""
create_consolidado_vendas_marca_linha = """
CREATE VIEW Tabela2 AS
SELECT id_marca,marca,id_linha, linha, SUM(qtd_vendas) AS total_de_vendas
FROM total_vendas 
GROUP BY marca 
ORDER BY `id_marca` ASC;"""

create_consolidado_vendas_marca_ano_mes = """
CREATE VIEW Tabela3 AS
SELECT id_marca,marca, SUM(qtd_vendas) AS total_de_vendas
FROM total_vendas 
GROUP BY marca 
ORDER BY `id_marca` ASC;"""

create_consolidado_vendas_linha_ano_mes = """
CREATE VIEW Tabela4 AS
SELECT id_linha,linha, SUM(qtd_vendas) AS total_de_vendas
FROM total_vendas 
GROUP BY marca 
ORDER BY total_de_vendas ASC;"""

connection = create_db_connection('127.0.0.1', "root", pw, db)
execute_query(connection, create_consolidado_vendas_ano_mes)
execute_query(connection, create_consolidado_vendas_marca_linha)
execute_query(connection, create_consolidado_vendas_marca_ano_mes)
execute_query(connection, create_consolidado_vendas_linha_ano_mes)

Conexão com banco Mysql concluída
Query executada
Query executada
Query executada
Query executada


# 5. Consumo de dados com python através da API do Twitter.

### 5.1 - Através do API do twitter criei um processo que salva os nomes dos usuários e o texto dos twitts.

Após criar uma conta de acesso comum ao Twitter, acessar https://developer.twitter.com/en/apply-for-access e criar uma conta 
de desenvolvedor para extraçao de dados baseado em parametros definidos e a partir disso, salvar os nomes e os twitts.

### 5.2 - Importação da biblioteca para fazer a conexão com a API.


In [ ]:
import tweepy as tw

### 5.3 - Após criado o app, acesse o mesmo e gere o token e token secret

In [ ]:
bearer_token ='AAAAAAAAAAAAAAAAAAAAAL3beQEAAAAAYz5B5pK%2FZKrYxOXejHhDVp3wNmI%3Dc2ljhc0SZp6wYWnSVif8HtIu58xZM9amFx2R4yWvj88s1shg2P'
consumer_key = 'ZlAdU2oj5V2PYrX7FCmHM0PQK'
consumer_secret = 'EA9WaibN9G5NmRMvljSkdcdSV80rlnBrYBWfDy5s7DLjKZ3uR4N'
access_token = '1543932014180564993-2yt2JjdxVKKNXzrr0Cau6BLiReBbo7'
access_token_secret = 'GFS4XODc0rWouILidXkFpccue2g39aXFCQVpuuNV29mjw'

In [35]:
#Criando o client 
cliente = tw.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret)

In [36]:
pesquisa = cliente.search_recent_tweets(query='Boticário',tweet_fields= ['lang'], max_results=50,expansions =['author_id'])

In [37]:
users = {u['id']:u for u in pesquisa.includes['users']}
lang = 'pt'

In [1]:
#Criaçao da tabela para os tweets
twitter_table = """ 
CREATE TABLE twitter_table(
    nome_usuario text,
    texto_twitts text);"""

connection = create_db_connection('127.0.0.1', "root", pw, db)
execute_query(connection, twitter_table)

### 5.4 - Leitura dos twitts linha por linha filtrando twitts e nomes

In [ ]:
for tweet in pesquisa.data:
    texto =tweet.text
    if users[tweet.author_id]:
        user = users[tweet.author_id]
        print(f'Usuário(a):{user.username}')
        if('RT' in texto):
            posicao = texto.find(':')
            texto = texto [posicao+2:]
            print(f'Mensagem:{texto}')
            
          
    dados = '('+'\''+str(user.name)+'\''+',\''+str(texto)+'\''+')'
    inserir_no_banco = """INSERT INTO `twitter_table`(`nome_usuario`, `texto_twitts`) VALUES"""
    comando = inserir_no_banco + dados
    try:
        execute_query(connection, comando)
    except:
        continue

            
connection = create_db_connection('127.0.0.1', "root", pw, db)
execute_query(connection, comando)